In [1]:
import chess
import chess.pgn
import ipywidgets as widgets
import chess.svg
from IPython.display import display, clear_output
import time
from datetime import datetime
from versions.my_engine_v1 import get_best_move_v1 as v1
from versions.my_engine_v2 import get_best_move_v2 as v2
from versions.my_engine_v3 import get_best_move_iterative as v3
from stockfish import Stockfish

# board = chess.Board(fen="r1bqk1r1/ppp2pp1/2n1p2p/3p2N1/2PPn2P/P3PP2/1P1B2P1/R2QKB1R b KQq - 0 2")
board = chess.Board()
AI_DEPTH = 3

In [2]:
sf = Stockfish(path="./stockfish/stockfish-windows-x86-64-avx2.exe")
def stock_fish(board,depth=AI_DEPTH):
    sf.set_depth(depth)
    sf.set_fen_position(board.fen())
    move = chess.Move.from_uci(sf.get_best_move())
    return move


In [3]:
# %% Board visualization helper

def show_board(board, last_move=None, engine_move=None, size=420):
    arrows = []
    if last_move:
        arrows.append(
            chess.svg.Arrow(
                last_move.from_square,
                last_move.to_square,
                color="#888888"
            )
        )
    if engine_move:
        arrows.append(
            chess.svg.Arrow(
                engine_move.from_square,
                engine_move.to_square,
                color="#00cc44"
            )
        )

    display(
        chess.svg.board(
            board,
            size=size,
            arrows=arrows,
            lastmove=last_move
        )
    )


In [4]:
def savepgn(game, file, game_no=False, mode="a"):
    if game_no is not False:
        data = f"\n{game}"
    else:
        data = str(game)

    if mode == "prepend":
        # Read old data, put new data in front, write everything back
        try:
            with open(file, "r") as f:
                old_data = f.read()
        except FileNotFoundError:
            old_data = ""
            
        with open(file, "w") as f:
            f.write(data + "\n\n" + old_data)
    else:
        # Standard append for normal games
        with open(file, "a") as f:
            print(data, file=f, end="\n\n")
            
    print(f"Saved to {file}")

In [5]:
def human(board,depth=AI_DEPTH):
    try:
        move=input("add move in uci format: ")
        return chess.Move.from_uci(move)
    except Exception as e:
        print("Error",e)
    pass

In [6]:
def play(count , white , black, show=True ,claim_draw=True,AI_DEPTH=AI_DEPTH, board=board,save=True):
    def timed_engine(engine_fn):
        def wrapper(board, depth):
            start = time.time()
            move = engine_fn(board, depth)
            elapsed = time.time() - start
            return move, elapsed
        # ✅ set metadata ONCE
        wrapper.__name__ = engine_fn.__name__
        wrapper.__qualname__ = engine_fn.__qualname__
        return wrapper
    
    black = timed_engine(black)
    white = timed_engine(white)
    file=f"games played/{datetime.now().strftime('%Y-%m-%d %H-%M-%S')}.pgn"
    white_won = black_won = draw = 0
    
    for i in range(count):
        
        board = chess.Board()

        game = chess.pgn.Game.from_board(board)
        game.headers["Event"] = f"Game {i+1}"
        game.headers["Site"] = "Local"
        game.headers["White"] = f"{white.__name__}"
        game.headers["Black"] = f"{black.__name__}"
        node = game 

        print(f"Playing game:- {i+1}/{count}")
        while not board.is_game_over(claim_draw=claim_draw):

            if board.turn == chess.WHITE:
                move,t = white(board, AI_DEPTH)
            else:
                move,t = black(board, AI_DEPTH)
                

            board.push(move)
            node = node.add_variation(move)

            if show:
                clear_output(wait=True)
                show_board(board,last_move=move)
                print(f"Playing game:- {i+1}/{count} {board.fullmove_number}")


        result = board.result(claim_draw=claim_draw)
        game.headers["Result"] = result
        savepgn(game,file,game_no=i) if save else ""

        if result == "1-0": white_won += 1
        elif result == "0-1": black_won += 1
        else:draw += 1

    summary = (
        f"Match Summary:\n"
        f"Total Games: {count}\n"
        f"White ({white.__name__}) Won: {white_won}\n"
        f"Black ({black.__name__}) Won: {black_won}\n"
        f"Draws: {draw}\n"
        f"----------------------------------------"
    )
    if save: savepgn(summary,file,mode="prepend")
    print(summary)
    return

In [7]:
# %% Instrumented play loop (cell-level only)

def play_debug(count, white, black, AI_DEPTH=3, show=True):
    white_won = black_won = draw = 0

    for i in range(count):
        board = chess.Board()
        print(f"\n=== Game {i+1}/{count} ===")

        move_no = 1
        while not board.is_game_over():
            if board.turn == chess.WHITE:
                move, t = v3_timed(board, AI_DEPTH)
                who = "ENGINE"
            else:
                move, t = stockfish_timed(board, AI_DEPTH)
                who = "STOCKFISH"

            board.push(move)

            clear_output(wait=True)
            show_board(board, last_move=move)
            print(
                f"{move_no}. {who} played {move.uci()} | "
                f"time {t:.2f}s | FEN {board.fen()}"
            )

            move_no += 1

        result = board.result()
        print("Result:", result)

        if result == "1-0": white_won += 1
        elif result == "0-1": black_won += 1
        else: draw += 1

    print("\nSUMMARY")
    print(f"White wins: {white_won}")
    print(f"Black wins: {black_won}")
    print(f"Draws: {draw}")


In [ ]:
play(count=5,white=v3,black=stock_fish,AI_DEPTH=3,save=False)
# from versions.my_engine_v3 import evaluate_board
# board=chess.Board(fen="7k/3R4/8/8/8/8/8/7K w - - 0 1")
# print(evaluate_board(board))
# TODO: fix last move 

black piece1 0
black piece -20
Eval: -20 , position score:20 Sqaure: 63 pice: k turn:True
Eval: 500 , position score:20 Sqaure: 51 pice: R turn:True
Eval: 520 , position score:20 Sqaure: 7 pice: K turn:True
520
